In [1]:
from datgan import DATGAN
import datgan

import numpy as np
import pandas as pd
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import tensorflow as tf
#tf.config.run_functions_eagerly(True)

In [2]:
df = pd.read_csv('../../data/LPMC/trips.csv', index_col=False)

In [3]:
df.head()

,travel_mode,purpose,faretype,day_of_week,start_time_linear,age,female,driving_license,distance,dur_walking,dur_cycling,dur_driving,driving_traffic_percent,hh_vehicles,hh_income,hh_people,dur_pt,hh_region
0,walk,HBO,full,6,15.833333,30,0,1,2145,0.553056,0.157500,0.158333,0.473684,0,35-50k,2,0.436389,Central London
1,pt,HBO,full,7,10.000000,50,1,1,1789,0.473333,0.160556,0.135000,0.547325,0,15-20k,5,0.271111,Central London
2,pt,HBW,full,5,17.000000,55,1,1,10036,2.411667,0.761389,0.638056,0.543317,2,75-100k,2,0.830833,South London
3,drive,HBO,full,6,16.883333,51,1,1,1531,0.423889,0.168611,0.110556,0.268844,1,>100k,2,0.295556,East London
4,pt,HBW,full,3,7.500000,39,1,1,1124,0.275833,0.123611,0.081667,0.156463,1,>100k,4,0.124722,South London


In [4]:
# First, define the specificities of continuous variables
data_info = {
    'start_time_linear': {
        'type': 'continuous',
        'bounds': [0.0, 23.999],
        'discrete': False,
    },
    'age': {
        'type': 'continuous',
        'bounds': [0, 100],
        'discrete': True
    },
    'distance': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'discrete': True,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_walking': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_cycling': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_pt': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_driving': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'driving_traffic_percent': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'discrete': False,
    },
}

# Add the other variables as categorical
for c in df.columns:
    if c not in data_info.keys():
        data_info[c] = {'type': 'categorical'}

In [5]:
df.columns

Index(['travel_mode', 'purpose', 'faretype', 'day_of_week',
       'start_time_linear', 'age', 'female', 'driving_license', 'distance',
       'dur_walking', 'dur_cycling', 'dur_driving', 'driving_traffic_percent',
       'hh_vehicles', 'hh_income', 'hh_people', 'dur_pt', 'hh_region'],
      dtype='object')

In [6]:
# personalised graph
graph = nx.DiGraph()

graph.add_edges_from([
    ('hh_region', 'hh_people'),
    ('hh_region', 'distance'),
    ('hh_region', 'hh_income'),
    ('hh_region', 'travel_mode'),
    ('hh_income', 'hh_vehicles'),
    ('hh_people', 'hh_vehicles'),
    ('age', 'hh_people'),
    ('age', 'faretype'),
    ('age', 'driving_license'),
    ('age', 'purpose'),
    ('age', 'travel_mode'),
    ('female', 'driving_license'),
    ('female', 'hh_people'),
    ('driving_license', 'travel_mode'),
    ('hh_vehicles', 'driving_license'),
    ('hh_vehicles', 'travel_mode'),
    ('faretype', 'travel_mode'),
    ('day_of_week', 'purpose'),
    ('day_of_week', 'start_time_linear'),
    ('day_of_week', 'driving_traffic_percent'),
    ('purpose', 'start_time_linear'),
    ('purpose', 'travel_mode'),
    ('purpose', 'distance'),
    ('start_time_linear', 'driving_traffic_percent'),
    ('driving_traffic_percent', 'dur_driving'),
    ('distance', 'driving_traffic_percent'),
    ('distance', 'dur_walking'),
    ('distance', 'dur_cycling'),
    ('distance', 'dur_pt'),
    ('distance', 'dur_driving'),
    ('distance', 'travel_mode')
])

In [7]:
name = 'DATGAN'

In [8]:
output_folder = '../output_region/{}/'.format(name)

In [9]:
datgan = DATGAN(output=output_folder,
                loss_function='WGGP',
                batch_size=1101,
                num_epochs=1000)

In [10]:
datgan.fit(df, data_info, graph, preprocessed_data_path='../output/encoded_LPMC')

Preprocessed data have been loaded!
Start training DATGAN with the WGGP loss (11/05/2022 12:20:26).
Restored models from epoch 1000.


Training DATGAN: 0it [00:00, ?it/s]

DATGAN has finished training (11/05/2022 12:20:26) - Training time: 00 second


In [11]:
for i in range(5):
    samp = datgan.sample(len(df))
    samp.to_csv('../../data/synthetic/regions/test/DATGAN_{:02d}.csv'.format(i+1), index=False)

Sampling from DATGAN: 100%|██████████| 16904/16904 [00:17<00:00, 978.66it/s] 


# Regions

In [11]:
nbrs = {}
dct = {}

for r in df.hh_region.unique():
    tmp = pd.read_csv('../../data/nomis/{}.csv'.format(r))

    nbrs[r] = len(tmp)
    dct[r] = []

In [12]:
remaining_regions = set(df.hh_region.unique())

count = 1
while remaining_regions:

    print("Pass {} - Remaining regions: {}".format(count, len(remaining_regions)))

    samp = datgan.sample(100000)

    region_to_remove = []

    for r in remaining_regions:
        tmp = samp[samp.hh_region == r]

        if len(tmp) > nbrs[r]:
            tmp = tmp.sample(nbrs[r], replace=False)

        nbrs[r] -= len(tmp)
        dct[r].append(tmp)

        if nbrs[r] == 0:
            region_to_remove.append(r)


    for r in region_to_remove:
        remaining_regions.remove(r)

    count += 1

Pass 1 - Remaining regions: 5


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:38<00:00, 1013.39it/s]


Pass 2 - Remaining regions: 5


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:35<00:00, 1045.48it/s]


Pass 3 - Remaining regions: 5


...

Sampling from DATGAN: 100%|██████████| 100000/100000 [01:34<00:00, 1062.20it/s]


Pass 84 - Remaining regions: 1


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:34<00:00, 1062.39it/s]


Pass 85 - Remaining regions: 1


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:34<00:00, 1062.39it/s]


In [13]:
for r in dct.keys():
    tmp = pd.concat(dct[r])
    tmp.to_csv('../../data/synthetic/regions/DATGAN/{}.csv'.format(r), index=False)

# Boroughs

In [11]:
nbrs = {}
dct = {}

for r in df.hh_borough.unique():
    tmp = pd.read_csv('../../data/nomis/{}.csv'.format(r))

    nbrs[r] = len(tmp)
    dct[r] = []

In [12]:
remaining_boroughs = set(df.hh_borough.unique())

count = 1
while remaining_boroughs:

    print("Pass {} - Remaning boroughs: {}".format(count, len(remaining_boroughs)))

    samp = datgan.sample(100000)

    borough_to_remove = []

    for r in remaining_boroughs:
        tmp = samp[samp.hh_borough == r]

        if len(tmp) > nbrs[r]:
            tmp = tmp.sample(nbrs[r], replace=False)

        nbrs[r] -= len(tmp)
        dct[r].append(tmp)

        if nbrs[r] == 0:
            borough_to_remove.append(r)


    for r in borough_to_remove:
        remaining_boroughs.remove(r)

    count += 1

Pass 1 - Remaning boroughs: 33


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:41<00:00, 981.84it/s]


Pass 2 - Remaning boroughs: 33


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:37<00:00, 1024.45it/s]


Pass 3 - Remaning boroughs: 33


...

Sampling from DATGAN: 100%|██████████| 100000/100000 [01:34<00:00, 1058.00it/s]


Pass 121 - Remaning boroughs: 1


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:34<00:00, 1059.11it/s]


Pass 122 - Remaning boroughs: 1


Sampling from DATGAN: 100%|██████████| 100000/100000 [01:34<00:00, 1060.92it/s]


In [13]:
for r in dct.keys():
    tmp = pd.concat(dct[r])
    tmp.to_csv('../../data/synthetic/boroughs/DATGAN/{}.csv'.format(r), index=False)